# Evaluación de Modelos de Redes Neuronales sobre Imágenes Termográficas para la Detección del Cáncer de Mama

## Tesis de Maestría en Inteligencia Artificial Aplicada

**Objetivo:** Evaluar modelos basados en redes neuronales profundas para la clasificación automática de imágenes termográficas en la detección temprana del cáncer de mama.

**Metodología:** Feature Extraction con CNNs preentrenadas + Clasificador SVM
- **Modelo 1:** EfficientNet-B0 (extractor) + SVM (clasificador)
- **Modelo 2:** ResNet50 (extractor) + SVM (clasificador)

**Dataset:** DMR-IR del Visual Lab UFF (272 imágenes: 177 healthy, 95 sick)

---
## 1. Configuración del Entorno

In [ ]:
# Importaciones estándar
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
import json

# Configurar warnings
warnings.filterwarnings('ignore')

# Agregar directorio src al path
sys.path.insert(0, str(Path.cwd().parent / 'src'))
sys.path.insert(0, str(Path.cwd().parent))

# Verificar PyTorch y dispositivo
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"MPS disponible: {torch.backends.mps.is_available()}")
print(f"CUDA disponible: {torch.cuda.is_available()}")

# Configurar dispositivo
if torch.backends.mps.is_available():
    DEVICE = torch.device("mps")
    print("Usando MPS (Apple Silicon GPU)")
elif torch.cuda.is_available():
    DEVICE = torch.device("cuda")
    print(f"Usando CUDA: {torch.cuda.get_device_name(0)}")
else:
    DEVICE = torch.device("cpu")
    print("Usando CPU")

# Configuración de reproducibilidad
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
torch.manual_seed(RANDOM_STATE)

In [ ]:
# Importar módulos del proyecto
from src import preprocessing, feature_extraction, models, evaluation, visualization

# Recargar módulos en caso de modificaciones
from importlib import reload
reload(preprocessing)
reload(feature_extraction)
reload(models)
reload(evaluation)
reload(visualization)

print("Módulos cargados correctamente")

In [ ]:
# Configuración de rutas
PROJECT_ROOT = Path.cwd().parent
DATA_DIR = PROJECT_ROOT / 'data' / 'raw'
RESULTS_DIR = PROJECT_ROOT / 'results'
FIGURES_DIR = RESULTS_DIR / 'figures'
MODELS_DIR = RESULTS_DIR / 'models'
METRICS_DIR = RESULTS_DIR / 'metrics'
FEATURES_DIR = RESULTS_DIR / 'features'

# Crear directorios si no existen
for directory in [FIGURES_DIR, MODELS_DIR, METRICS_DIR, FEATURES_DIR]:
    directory.mkdir(parents=True, exist_ok=True)

print(f"Directorio del proyecto: {PROJECT_ROOT}")
print(f"Directorio de datos: {DATA_DIR}")

---
## 2. Configuración del Dataset

**IMPORTANTE:** Configura la ruta a tu dataset DMR-IR en la siguiente celda.

In [ ]:
# ============================================
# CONFIGURA AQUÍ LA RUTA A TU DATASET
# ============================================
# El dataset debe tener la estructura:
# DMR-IR/
# ├── healthy/
# │   ├── imagen1.png
# │   └── ...
# └── sick/
#     ├── imagen1.png
#     └── ...

# Opción 1: Ruta absoluta
# DATASET_PATH = Path("/ruta/completa/a/DMR-IR")

# Opción 2: Copiar dataset a data/raw/DMR-IR
DATASET_PATH = DATA_DIR / 'DMR-IR'

# Verificar existencia
if not DATASET_PATH.exists():
    print(f"⚠️  ADVERTENCIA: El dataset no se encontró en: {DATASET_PATH}")
    print("\nPor favor:")
    print("1. Copia el dataset DMR-IR a la carpeta 'data/raw/'")
    print("2. O modifica DATASET_PATH con la ruta correcta")
else:
    print(f"✓ Dataset encontrado en: {DATASET_PATH}")

In [ ]:
# Obtener información del dataset
dataset_info = preprocessing.get_dataset_info(DATASET_PATH)

print("="*50)
print("INFORMACIÓN DEL DATASET")
print("="*50)
print(f"Total de imágenes: {dataset_info['total_count']}")
print(f"  - Healthy (sanas): {dataset_info['healthy_count']}")
print(f"  - Sick (enfermas): {dataset_info['sick_count']}")
print(f"\nRatio de desbalance: {dataset_info['healthy_count']/dataset_info['sick_count']:.2f}:1")

---
## 3. Carga y Preprocesamiento del Dataset

In [ ]:
# Cargar y preprocesar todas las imágenes
# Este proceso aplica:
# 1. CLAHE para normalización de contraste
# 2. Redimensionado a 224x224
# 3. Normalización ImageNet

print("Cargando y preprocesando imágenes...")
print("(Este proceso puede tardar unos minutos)\n")

images, labels, paths = preprocessing.load_dataset(
    DATASET_PATH,
    target_size=(224, 224),
    apply_clahe_norm=True,
    normalize_imagenet=True
)

print(f"\nImágenes cargadas: {images.shape}")
print(f"Labels: {labels.shape}")
print(f"Distribución: Healthy={sum(labels==0)}, Sick={sum(labels==1)}")

In [ ]:
# Visualizar distribución de clases
fig = visualization.plot_class_distribution(
    labels,
    split_name="Complete Dataset",
    output_dir=str(FIGURES_DIR)
)
plt.show()

---
## 4. División del Dataset (Train/Val/Test)

In [ ]:
# División estratificada: 70% train, 15% validation, 15% test
# CRÍTICO: La división se hace ANTES de cualquier augmentation

splits = preprocessing.get_data_splits(
    images, labels, paths,
    test_size=0.30,  # 30% para val+test
    val_ratio=0.50,  # 50% de ese 30% para test (15% del total)
    random_state=RANDOM_STATE
)

# Extraer los splits
X_train = splits['X_train']
X_val = splits['X_val']
X_test = splits['X_test']
y_train = splits['y_train']
y_val = splits['y_val']
y_test = splits['y_test']

In [ ]:
# Visualizar distribución en cada split
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for ax, (name, y) in zip(axes, [('Train', y_train), ('Validation', y_val), ('Test', y_test)]):
    counts = [sum(y == 0), sum(y == 1)]
    bars = ax.bar(['Healthy', 'Sick'], counts, color=['#27ae60', '#e74c3c'])
    ax.set_title(f'{name} Set (n={len(y)})')
    ax.set_ylabel('Count')
    for bar, count in zip(bars, counts):
        ax.annotate(f'{count}', xy=(bar.get_x() + bar.get_width()/2, bar.get_height()),
                   xytext=(0, 3), textcoords='offset points', ha='center')

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'dataset_splits_distribution.png', dpi=300)
plt.show()

---
## 5. Visualización de Ejemplos de Preprocesamiento

In [ ]:
# Cargar algunas imágenes sin preprocesamiento para comparación
import cv2

# Seleccionar ejemplos (2 healthy, 2 sick)
example_indices = [
    np.where(labels == 0)[0][0],  # Primera healthy
    np.where(labels == 0)[0][1],  # Segunda healthy
    np.where(labels == 1)[0][0],  # Primera sick
    np.where(labels == 1)[0][1],  # Segunda sick
]

original_images = []
for idx in example_indices:
    img = cv2.imread(paths[idx])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    original_images.append(img.astype(np.float32) / 255.0)

processed_images = [images[idx] for idx in example_indices]
example_labels = [labels[idx] for idx in example_indices]

# Visualizar
fig = visualization.plot_preprocessing_examples(
    original_images,
    processed_images,
    example_labels,
    n_examples=4,
    output_dir=str(FIGURES_DIR)
)
plt.show()

---
## 6. Extracción de Características con EfficientNet-B0

In [ ]:
print("="*60)
print("EXTRACCIÓN DE CARACTERÍSTICAS CON EFFICIENTNET-B0")
print("="*60)

# Extraer features del conjunto de entrenamiento (con augmentation)
print("\n[1/3] Extrayendo features de entrenamiento (con augmentation)...")
features_train_effnet, labels_train_aug = feature_extraction.extract_features_with_augmentation(
    X_train, y_train,
    model_name='efficientnet_b0',
    device=DEVICE,
    batch_size=32,
    num_augmentations=5
)

# Extraer features de validación (sin augmentation)
print("\n[2/3] Extrayendo features de validación...")
features_val_effnet = feature_extraction.extract_features_simple(
    X_val,
    model_name='efficientnet_b0',
    device=DEVICE,
    batch_size=32
)

# Extraer features de test (sin augmentation)
print("\n[3/3] Extrayendo features de test...")
features_test_effnet = feature_extraction.extract_features_simple(
    X_test,
    model_name='efficientnet_b0',
    device=DEVICE,
    batch_size=32
)

print(f"\nFeatures extraídas:")
print(f"  Train: {features_train_effnet.shape}")
print(f"  Val:   {features_val_effnet.shape}")
print(f"  Test:  {features_test_effnet.shape}")

In [ ]:
# Guardar features de EfficientNet
np.save(FEATURES_DIR / 'efficientnet_features_train.npy', features_train_effnet)
np.save(FEATURES_DIR / 'efficientnet_features_val.npy', features_val_effnet)
np.save(FEATURES_DIR / 'efficientnet_features_test.npy', features_test_effnet)
np.save(FEATURES_DIR / 'efficientnet_labels_train_aug.npy', labels_train_aug)
print("Features de EfficientNet guardadas")

---
## 7. Entrenamiento de SVM con Features de EfficientNet-B0

In [ ]:
print("="*60)
print("ENTRENAMIENTO DE SVM CON FEATURES DE EFFICIENTNET-B0")
print("="*60)

# Entrenar SVM con búsqueda de hiperparámetros
model_effnet_svm, best_params_effnet, cv_results_effnet = models.train_svm_classifier(
    features_train_effnet,
    labels_train_aug,
    features_val_effnet,
    y_val,
    cv_folds=5,
    scoring='f1',
    n_jobs=-1
)

print(f"\nMejores hiperparámetros encontrados:")
for param, value in best_params_effnet.items():
    print(f"  {param}: {value}")

In [ ]:
# Evaluar en el conjunto de test
print("\nEvaluación en conjunto de TEST:")
metrics_effnet, y_pred_effnet, y_proba_effnet = evaluation.evaluate_model(
    model_effnet_svm,
    features_test_effnet,
    y_test,
    model_name="EfficientNet-B0 + SVM"
)

# Guardar modelo
models.save_model(
    model_effnet_svm,
    str(MODELS_DIR / 'efficientnet_svm.joblib'),
    metadata={
        'model_name': 'EfficientNet-B0 + SVM',
        'best_params': best_params_effnet,
        'metrics': metrics_effnet
    }
)

In [ ]:
# Matriz de confusión para EfficientNet-B0 + SVM
fig_cm_effnet = visualization.plot_confusion_matrix(
    y_test, y_pred_effnet,
    model_name="EfficientNet-B0 + SVM",
    output_dir=str(FIGURES_DIR)
)
plt.show()

---
## 8. Extracción de Características con ResNet50

In [ ]:
print("="*60)
print("EXTRACCIÓN DE CARACTERÍSTICAS CON RESNET50")
print("="*60)

# Extraer features del conjunto de entrenamiento (con augmentation)
print("\n[1/3] Extrayendo features de entrenamiento (con augmentation)...")
features_train_resnet, labels_train_aug_resnet = feature_extraction.extract_features_with_augmentation(
    X_train, y_train,
    model_name='resnet50',
    device=DEVICE,
    batch_size=32,
    num_augmentations=5
)

# Extraer features de validación (sin augmentation)
print("\n[2/3] Extrayendo features de validación...")
features_val_resnet = feature_extraction.extract_features_simple(
    X_val,
    model_name='resnet50',
    device=DEVICE,
    batch_size=32
)

# Extraer features de test (sin augmentation)
print("\n[3/3] Extrayendo features de test...")
features_test_resnet = feature_extraction.extract_features_simple(
    X_test,
    model_name='resnet50',
    device=DEVICE,
    batch_size=32
)

print(f"\nFeatures extraídas:")
print(f"  Train: {features_train_resnet.shape}")
print(f"  Val:   {features_val_resnet.shape}")
print(f"  Test:  {features_test_resnet.shape}")

In [ ]:
# Guardar features de ResNet
np.save(FEATURES_DIR / 'resnet_features_train.npy', features_train_resnet)
np.save(FEATURES_DIR / 'resnet_features_val.npy', features_val_resnet)
np.save(FEATURES_DIR / 'resnet_features_test.npy', features_test_resnet)
np.save(FEATURES_DIR / 'resnet_labels_train_aug.npy', labels_train_aug_resnet)
print("Features de ResNet guardadas")

---
## 9. Entrenamiento de SVM con Features de ResNet50

In [ ]:
print("="*60)
print("ENTRENAMIENTO DE SVM CON FEATURES DE RESNET50")
print("="*60)

# Entrenar SVM con búsqueda de hiperparámetros
model_resnet_svm, best_params_resnet, cv_results_resnet = models.train_svm_classifier(
    features_train_resnet,
    labels_train_aug_resnet,
    features_val_resnet,
    y_val,
    cv_folds=5,
    scoring='f1',
    n_jobs=-1
)

print(f"\nMejores hiperparámetros encontrados:")
for param, value in best_params_resnet.items():
    print(f"  {param}: {value}")

In [ ]:
# Evaluar en el conjunto de test
print("\nEvaluación en conjunto de TEST:")
metrics_resnet, y_pred_resnet, y_proba_resnet = evaluation.evaluate_model(
    model_resnet_svm,
    features_test_resnet,
    y_test,
    model_name="ResNet50 + SVM"
)

# Guardar modelo
models.save_model(
    model_resnet_svm,
    str(MODELS_DIR / 'resnet_svm.joblib'),
    metadata={
        'model_name': 'ResNet50 + SVM',
        'best_params': best_params_resnet,
        'metrics': metrics_resnet
    }
)

In [ ]:
# Matriz de confusión para ResNet50 + SVM
fig_cm_resnet = visualization.plot_confusion_matrix(
    y_test, y_pred_resnet,
    model_name="ResNet50 + SVM",
    output_dir=str(FIGURES_DIR)
)
plt.show()

---
## 10. Comparación de Modelos

In [ ]:
# Preparar datos para comparación
all_metrics = {
    'EfficientNet-B0 + SVM': metrics_effnet,
    'ResNet50 + SVM': metrics_resnet
}

# Tabla comparativa
print("="*70)
print("TABLA COMPARATIVA DE RESULTADOS")
print("="*70)
comparison_table = evaluation.compare_models(all_metrics)
print(comparison_table)

In [ ]:
# Gráfico de comparación de métricas
fig_metrics = visualization.plot_metrics_comparison(
    all_metrics,
    metrics_to_plot=['accuracy', 'precision', 'recall', 'specificity', 'f1_score'],
    output_dir=str(FIGURES_DIR)
)
plt.show()

In [ ]:
# Curvas ROC comparativas
from sklearn.metrics import roc_curve, roc_auc_score

# Calcular datos ROC
fpr_effnet, tpr_effnet, _ = roc_curve(y_test, y_proba_effnet)
auc_effnet = roc_auc_score(y_test, y_proba_effnet)

fpr_resnet, tpr_resnet, _ = roc_curve(y_test, y_proba_resnet)
auc_resnet = roc_auc_score(y_test, y_proba_resnet)

roc_data = {
    'EfficientNet-B0 + SVM': (fpr_effnet, tpr_effnet, auc_effnet),
    'ResNet50 + SVM': (fpr_resnet, tpr_resnet, auc_resnet)
}

fig_roc = visualization.plot_roc_curves(
    roc_data,
    output_dir=str(FIGURES_DIR)
)
plt.show()

---
## 11. Visualización de Distribución de Features

In [ ]:
# Visualizar distribución de features con t-SNE (EfficientNet)
print("Generando visualización t-SNE para EfficientNet-B0...")
fig_tsne_effnet = visualization.plot_feature_distribution(
    features_test_effnet,
    y_test,
    method='tsne',
    model_name="EfficientNet-B0",
    output_dir=str(FIGURES_DIR)
)
plt.show()

In [ ]:
# Visualizar distribución de features con t-SNE (ResNet)
print("Generando visualización t-SNE para ResNet50...")
fig_tsne_resnet = visualization.plot_feature_distribution(
    features_test_resnet,
    y_test,
    method='tsne',
    model_name="ResNet50",
    output_dir=str(FIGURES_DIR)
)
plt.show()

---
## 12. Guardar Resultados Finales

In [ ]:
# Guardar todas las métricas en JSON
all_results = {
    'efficientnet_b0_svm': {
        'metrics': {k: float(v) if isinstance(v, (np.floating, np.integer)) else v 
                   for k, v in metrics_effnet.items()},
        'best_params': best_params_effnet
    },
    'resnet50_svm': {
        'metrics': {k: float(v) if isinstance(v, (np.floating, np.integer)) else v 
                   for k, v in metrics_resnet.items()},
        'best_params': best_params_resnet
    }
}

with open(METRICS_DIR / 'comparison_results.json', 'w') as f:
    json.dump(all_results, f, indent=2)

print(f"Resultados guardados en: {METRICS_DIR / 'comparison_results.json'}")

In [ ]:
# Guardar labels de test para reproducibilidad
np.save(FEATURES_DIR / 'y_test.npy', y_test)
np.save(FEATURES_DIR / 'y_train.npy', y_train)
np.save(FEATURES_DIR / 'y_val.npy', y_val)
print("Labels guardadas")

In [ ]:
# Generar tabla LaTeX para la tesis
latex_table = visualization.create_latex_table(
    all_metrics,
    caption="Comparación de resultados entre modelos de clasificación",
    label="tab:model_comparison"
)

print("="*70)
print("TABLA LATEX PARA LA TESIS")
print("="*70)
print(latex_table)

# Guardar tabla LaTeX
with open(METRICS_DIR / 'results_table.tex', 'w') as f:
    f.write(latex_table)
print(f"\nTabla guardada en: {METRICS_DIR / 'results_table.tex'}")

---
## 13. Resumen Final

In [ ]:
print("="*70)
print("RESUMEN FINAL DE EXPERIMENTOS")
print("="*70)

print(f"\nDataset: DMR-IR")
print(f"Total de imágenes: {len(labels)}")
print(f"  - Healthy: {sum(labels==0)}")
print(f"  - Sick: {sum(labels==1)}")

print(f"\nDivisión del dataset:")
print(f"  - Train: {len(y_train)} ({len(y_train)/len(labels)*100:.1f}%)")
print(f"  - Validation: {len(y_val)} ({len(y_val)/len(labels)*100:.1f}%)")
print(f"  - Test: {len(y_test)} ({len(y_test)/len(labels)*100:.1f}%)")

print(f"\n" + "="*70)
print("RESULTADOS EN CONJUNTO DE TEST")
print("="*70)

# Determinar mejor modelo
best_model = 'EfficientNet-B0 + SVM' if metrics_effnet['f1_score'] > metrics_resnet['f1_score'] else 'ResNet50 + SVM'

for model_name, metrics in all_metrics.items():
    winner = " ⭐ MEJOR" if model_name == best_model else ""
    print(f"\n{model_name}{winner}")
    print("-" * 40)
    print(f"  Accuracy:    {metrics['accuracy']*100:.2f}%")
    print(f"  Precision:   {metrics['precision']*100:.2f}%")
    print(f"  Recall:      {metrics['recall']*100:.2f}%")
    print(f"  Specificity: {metrics['specificity']*100:.2f}%")
    print(f"  F1-Score:    {metrics['f1_score']*100:.2f}%")
    print(f"  AUC-ROC:     {metrics['auc_roc']:.4f}")

print(f"\n" + "="*70)
print("ARCHIVOS GENERADOS")
print("="*70)
print(f"\nFiguras: {FIGURES_DIR}")
print(f"Modelos: {MODELS_DIR}")
print(f"Métricas: {METRICS_DIR}")
print(f"Features: {FEATURES_DIR}")

---
## 14. Conclusiones

### Análisis de Resultados

Basado en los experimentos realizados, podemos concluir:

1. **Comparación de Arquitecturas:**
   - [Completar basado en resultados]

2. **Métricas Clínicas Relevantes:**
   - **Recall (Sensibilidad):** Mide la capacidad de detectar pacientes enfermos. Crítico para no perder casos.
   - **Specificity (Especificidad):** Mide la capacidad de identificar pacientes sanos. Reduce falsos positivos.

3. **Recomendación:**
   - [Completar basado en el mejor modelo]

### Limitaciones
- Dataset pequeño (272 imágenes)
- Posible sesgo por paletas de color diferentes

### Trabajo Futuro
- Validación con datasets externos
- Experimentar con fine-tuning de las CNNs
- Implementar técnicas de explicabilidad (Grad-CAM)